# Assignment: Preparing Data for Analysis (Modified Titanic)

![](https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/%20files/hw.png)

# 1) Load data & review the data

How many rows are there in the "titanic_to_student.csv"?

In [41]:
#Import the libraries
%pip install numpy
%pip install pandas
import numpy as np
import pandas as pd

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [89]:
df = pd.read_csv('titanic_to_student.csv')
print(df.shape)
df.head(5)

(445, 13)


,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,2,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0,PC 17599,71.2833,C85,C
1,1,4,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0,113803,53.1000,C123,S
2,2,6,0.0,3.0,"Moran, Mr. James",male,NaN,0.0,0,330877,8.4583,NaN,Q
3,3,8,0.0,3.0,"Palsson, Master. Gosta Leonard",male,2.0,3.0,1,349909,21.0750,NaN,S
4,4,10,1.0,2.0,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1.0,0,237736,30.0708,NaN,C


# 2) Drop unqualified variables

Drop unqualified variables

Drop variables with missing > 50%

Drop categorical variables with flat values > 70% (variables with the same value in the same column)

How many columns do we have left?





In [76]:

drop_col = list(df.columns[df.isna().mean() > 0.5])
for col in df.columns:
    unq_val = df[col].unique()
    col_cnt = df[col].value_counts(normalize=True)
    if col_cnt[col_cnt > 0.7].any():
        drop_col.append(col)

drop_df = df.drop(columns=drop_col)

print(drop_df.shape[0])

# df['Sex'].value_counts()


# print(df['Sex'].unique)

# drop_df = df.drop(columns=['Cabin'])
# Parch, Embarked

432


# 3) Remove all rows with missing target (the variable "Survived")

Remove all rows with missing targets (the variable "Survived")

How many rows do we have left?


In [44]:
drop_df.dropna(subset=['Survived'], inplace=True)

# 4) Handle outliers 
Handle outliers

For the variable “Fare”, replace outlier values with the boundary values

If value < (Q1 - 1.5IQR), replace with (Q1 - 1.5IQR)

If value > (Q3 + 1.5IQR), replace with (Q3 + 1.5IQR)

What is the average (mean) of “Fare” after replacing the outliers (round 2 decimal points)?

Hint: Use function round(_, 2)



In [45]:
fare_df = drop_df['Fare']

q1 = fare_df.quantile(0.25)
q3 = fare_df.quantile(0.75)
iqr = q3-q1

lower_bound = q1 - 1.5*iqr
upper_bound = q3 + 1.5*iqr

fare_df = np.where(
    fare_df < lower_bound, lower_bound, fare_df
)

fare_df = np.where(
    fare_df > upper_bound, upper_bound, fare_df
)

print(round(drop_df['Fare'].mean(), 2))
print(round(fare_df.mean(), 2))
drop_df['Fare'] = fare_df


34.5
26.23


# 5) Impute missing value

Impute missing value

For number type column, impute missing values with mean

What is the average (mean) of “Age” after imputing the missing values (round 2 decimal points)?

Hint: Use function round(_, 2)


In [62]:
mean_age = df['Age'].mean()
new_age = df['Age'].fillna(mean_age)
print(round(new_age.mean(), 2))

29.14


# 6) Convert categorical to numeric values

Convert categorical to numeric values

For the variable “Embarked”, perform the dummy coding.

What is the average (mean) of “Embarked_Q” after performing dummy coding (round 2 decimal points)?

Hint: Use function round(_, 2)



In [69]:
# Perform dummy coding
dummies = pd.get_dummies(df['Embarked'], prefix='Embarked')

# Add the dummy variables to the original DataFrame
new_df = pd.concat([df, dummies], axis=1)

# Calculate the mean of "Embarked_Q" and round to 2 decimal places
mean_embarked_q = round(new_df['Embarked_Q'].mean(), 2)
print(mean_embarked_q)


0.06


# 7) Partition data
Split train/test split with stratification using 70%:30% and random seed with 123

Show a proportion between survived (1) and died (0) in all data sets (total data, train, test)

What is the proportion of survivors (survived = 1) in the training data (round 2 decimal points)?

Hint: Use function round(_, 2), and train_test_split() from sklearn.model_selection




In [92]:
# %pip install scikit-learn
from sklearn.model_selection import train_test_split

# drop_df.dropna(subset=['Survived'], inplace=True)

mode_sur = 1.0 if df['Survived'].mean() > 0.5 else 0.0
new_df = df
new_df['Survived'] = df['Survived'].fillna(mode_sur)

X = new_df.drop('Survived', axis=1)  # Features
y = new_df['Survived']               # Target variable

# Perform stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=123, stratify=y
)

# Combine the features and target to see the full train/test splits
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

# Proportions of survivors in all datasets
total_proportion = df['Survived'].mean()
train_proportion = y_train.mean()
test_proportion = y_test.mean()

# Proportion of survivors (1) in the training data
train_survived_proportion = round(train_proportion, 2)

# Show the proportions
print(f"Proportion of survivors (1) in the total data: {total_proportion:.2f}")
print(f"Proportion of survivors (1) in the training data: {train_survived_proportion}")
print(f"Proportion of survivors (1) in the testing data: {test_proportion:.2f}")

Proportion of survivors (1) in the total data: 0.40
Proportion of survivors (1) in the training data: 0.41
Proportion of survivors (1) in the testing data: 0.40
